In [ ]:
import pandas as pd
import os

# Path to the main test.csv
main_test_csv = r"C:\Users\zezom\PycharmProjects\HorusEye\test.csv"

# Output folders (make sure these exist)
mi_dir = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI"
ssvep_dir = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\SSVEP"
os.makedirs(mi_dir, exist_ok=True)
os.makedirs(ssvep_dir, exist_ok=True)

# Load and clean columns
df = pd.read_csv(main_test_csv)
df.columns = df.columns.str.strip()

# Make sure the paradigm column exists
if 'paradigm' not in df.columns:
    raise ValueError("No 'paradigm' column in test.csv!")

# Split by paradigm
mi_df = df[df['paradigm'].str.upper() == "MI"]
ssvep_df = df[df['paradigm'].str.upper() == "SSVEP"]

# Save splits
mi_df.to_csv(os.path.join(mi_dir, "test.csv"), index=False)
ssvep_df.to_csv(os.path.join(ssvep_dir, "test.csv"), index=False)

print("✅ test.csv split into MI and SSVEP test sets!")


In [ ]:
import pandas as pd
df = pd.read_csv(r"C:\Users\zezom\PycharmProjects\HorusEye\test.csv")
print(df.columns)
print(df.head())


In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
from scipy.signal import welch
from tqdm import tqdm

# --- SETTINGS: SELECT TASK ---
MODE = "SSVEP"  # or "MI"

test_csv_path = r"C:\Users\zezom\PycharmProjects\HorusEye\test.csv"
submission_path = r"C:\Users\zezom\PycharmProjects\HorusEye/submission.csv"
samples_per_trial = 1750
sampling_rate = 250

if MODE == "SSVEP":
    csv_data_dir = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\SSVEP"
    svm_model_path = r"C:\Users\zezom\PycharmProjects\HorusEye/svm_ssvep_model.pkl"
    encoder_path = r"C:\Users\zezom\PycharmProjects\HorusEye/svm_ssvep_label_encoder.pkl"
    channels = None  # Use all columns
    psd_band = (5, 45)
elif MODE == "MI":
    csv_data_dir = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI"
    svm_model_path = r"C:\Users\zezom\PycharmProjects\HorusEye/svm_mi_model.pkl"
    encoder_path = r"C:\Users\zezom\PycharmProjects\HorusEye/svm_mi_label_encoder.pkl"
    channels = ['C3', 'C4']  # Use only MI channels
    mi_bands = ['mu', 'beta']  # Expected per your MI SVM code
else:
    raise ValueError("MODE must be 'SSVEP' or 'MI'")

# --- Load Model and Encoder ---
with open(svm_model_path, "rb") as f:
    svm = pickle.load(f)
with open(encoder_path, "rb") as f:
    label_encoder = pickle.load(f)

# --- Load Test Metadata ---
test_df = pd.read_csv(test_csv_path)
print(f"[INFO] Loaded {len(test_df)} test trials")

results = []

for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    trial_id = row['id']
    subject = str(row['subject_id']).strip()
    session = str(row['trial_session']).strip()
    trial_num = int(row['trial'])

    if MODE == "SSVEP":
        csv_path = os.path.join(csv_data_dir, f"{subject}_{session}_EEGdata_preprocessed.csv")
        if not os.path.exists(csv_path):
            print(f"[MISSING] {csv_path} for trial {trial_id}")
            results.append((trial_id, 'UNKNOWN'))
            continue
        try:
            df = pd.read_csv(csv_path)
            start_idx = (trial_num - 1) * samples_per_trial
            end_idx = trial_num * samples_per_trial
            trial_data = df.iloc[start_idx:end_idx].values
            if trial_data.shape[0] != samples_per_trial:
                print(f"[SKIP] {csv_path} trial {trial_id} - insufficient samples")
                results.append((trial_id, 'UNKNOWN'))
                continue
            # --- PSD Extraction (as in training) ---
            psd_features = []
            for ch in range(trial_data.shape[1]):
                freqs, psd = welch(trial_data[:, ch], fs=sampling_rate, nperseg=256)
                band = (freqs >= psd_band[0]) & (freqs <= psd_band[1])
                psd_features.extend(psd[band])
            X_test_feat = np.array(psd_features).reshape(1, -1)
            pred_idx = svm.predict(X_test_feat)[0]
            pred_label = label_encoder.inverse_transform([pred_idx])[0]
            results.append((trial_id, pred_label))
        except Exception as e:
            print(f"[ERROR] {csv_path} trial {trial_id}: {e}")
            results.append((trial_id, 'UNKNOWN'))

    elif MODE == "MI":
        # For MI, loop over channels/bands and reconstruct feature vector
        feature_vec = []
        incomplete = False
        for ch in channels:
            for band in mi_bands:
                mi_band_path = os.path.join(csv_data_dir, f"{subject}_{session}_EEGdata_preprocessed_{ch}_{band}.csv")
                if not os.path.exists(mi_band_path):
                    print(f"[MISSING] {mi_band_path} for trial {trial_id}")
                    incomplete = True
                    break
                try:
                    df = pd.read_csv(mi_band_path)
                    start_idx = (trial_num - 1) * samples_per_trial
                    end_idx = trial_num * samples_per_trial
                    trial_data = df.iloc[start_idx:end_idx].values
                    if trial_data.shape[0] != samples_per_trial:
                        print(f"[SKIP] {mi_band_path} trial {trial_id} - insufficient samples")
                        incomplete = True
                        break
                    feature_vec.extend(trial_data.flatten())
                except Exception as e:
                    print(f"[ERROR] {mi_band_path} trial {trial_id}: {e}")
                    incomplete = True
                    break
            if incomplete:
                break
        if incomplete or not feature_vec:
            results.append((trial_id, 'UNKNOWN'))
            continue
        try:
            X_test_feat = np.array(feature_vec).reshape(1, -1)
            pred_idx = svm.predict(X_test_feat)[0]
            pred_label = label_encoder.inverse_transform([pred_idx])[0]
            results.append((trial_id, pred_label))
        except Exception as e:
            print(f"[ERROR][MI] Prediction failed for {trial_id}: {e}")
            results.append((trial_id, 'UNKNOWN'))

# --- Make submission DataFrame ---
submit_df = pd.DataFrame(results, columns=['id', 'label'])
submit_df.to_csv(submission_path, index=False)
print(f"[INFO] Submission saved to {submission_path}")


In [ ]:
import pandas as pd

submission_path = r"C:\Users\zezom\PycharmProjects\HorusEye\submission.csv"
df = pd.read_csv(submission_path)

# Overview
print(df['label'].value_counts())

# Check if any UNKNOWN remain
unknowns = df[df['label'] == 'UNKNOWN']
print(f"Total UNKNOWN: {len(unknowns)}")
if not unknowns.empty:
    print(unknowns[['id']])


In [ ]:
print(f"[MISSING FILE] {csv_path}")


# Fixing

In [49]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# --- SETTINGS ---
model_path = r"C:\Users\zezom\PycharmProjects\HorusEye\best_mi_cnn_model.keras"
data_path = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI\S38_1_EEGdata_preprocessed.csv"
channels = ['C3_ref', 'CZ_ref', 'C4_ref']
samples_per_trial = 1750
trial_number = 1  # ⬅️ Change this to select a trial

# --- Load Pre-trained CNN Model ---
model = load_model(model_path)
print("[INFO] CNN model loaded.")

# --- Load EEG Data for Selected Trial ---
df = pd.read_csv(data_path)
start = (trial_number - 1) * samples_per_trial
end = trial_number * samples_per_trial

# Extract trial segment
trial_data = df[channels].iloc[start:end].values.T  # (channels, samples)

# --- Verify Sample Length ---
if trial_data.shape[1] != samples_per_trial:
    raise ValueError(f"[ERROR] Trial {trial_number} is incomplete: {trial_data.shape}")

# --- Normalize EEG Trial (Z-score) ---
X_trial = (trial_data - np.mean(trial_data, axis=-1, keepdims=True)) / (np.std(trial_data, axis=-1, keepdims=True) + 1e-6)
X_trial = X_trial[np.newaxis, ...]  # (1, channels, samples)

# --- Predict ---
y_prob = model.predict(X_trial)
y_pred = np.argmax(y_prob, axis=1)[0]

# --- Decode Label ---
label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(['Left', 'Right'])  # Make sure this matches training
predicted_label = label_encoder.inverse_transform([y_pred])[0]

print(f"[RESULT] CNN Prediction for Trial {trial_number}: {predicted_label}")


[INFO] CNN model loaded.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[RESULT] CNN Prediction for Trial 1: Left


In [3]:
print("Expected input shape:", model.input_shape)


Expected input shape: (None, 4, 1750)


In [38]:
import pandas as pd
import numpy as np
import pickle
from scipy.signal import welch

# Paths and model
csv_path = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\SSVEP\S40_1_EEGdata_preprocessed.csv"
model_path = r"C:\Users\zezom\PycharmProjects\HorusEye\svm_ssvep_model.pkl"
encoder_path = r"C:\Users\zezom\PycharmProjects\HorusEye\svm_ssvep_label_encoder.pkl"

with open(model_path, "rb") as f:
    ssvep_svm = pickle.load(f)
with open(encoder_path, "rb") as f:
    ssvep_label_encoder = pickle.load(f)

samples_per_trial = 1750
sampling_rate = 250
psd_band = (5, 45)

def get_ssvep_psd_features(trial_number):
    df = pd.read_csv(csv_path)
    start_idx = (trial_number - 1) * samples_per_trial
    end_idx = trial_number * samples_per_trial
    trial_data = df.iloc[start_idx:end_idx].values
    if trial_data.shape[0] != samples_per_trial:
        raise ValueError(f"Trial {trial_number} has insufficient samples")
    psd_features = []
    for ch in range(trial_data.shape[1]):
        freqs, psd = welch(trial_data[:, ch], fs=sampling_rate, nperseg=256)
        band = (freqs >= psd_band[0]) & (freqs <= psd_band[1])
        psd_features.extend(psd[band])
    return np.array(psd_features).reshape(1, -1)

TRIAL_NUMBER = 10 # Change as needed
X_test = get_ssvep_psd_features(TRIAL_NUMBER)
y_pred_idx = ssvep_svm.predict(X_test)[0]
y_pred_label = ssvep_label_encoder.inverse_transform([y_pred_idx])[0]

print(f"SSVEP prediction for trial {TRIAL_NUMBER}: {y_pred_label}")


[INFO] CNN-LSTM model loaded.
[INFO] Loaded DL-ready test data shape: (0, 8, 1750)
[INFO] Trials in file: 0


ValueError: Shape mismatch! Got (0, 8, 1750). Expecting (10, 4, 1750) for 10 trials.

In [72]:
df = pd.read_csv(csv_path)
print(f"Total rows in CSV: {len(df)}")
print(f"Expected rows for one trial: {samples_per_trial}")
print(f"Number of full trials: {len(df) // samples_per_trial}")


Total rows in CSV: 44
Expected rows for one trial: 1750
Number of full trials: 0


In [74]:
import pandas as pd
df = pd.read_csv(r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\SSVEP\S36_1_EEGdata_preprocessed.csv")
print(df.shape[0])  # Should be a multiple of 1750


44
